
### Step 1: PDF Processing

In [ ]:
!pip install pdfplumber

In [4]:
import pdfplumber
import json
import re
from pathlib import Path
from typing import Dict, List, Optional
from collections import defaultdict

BASE_DIR = Path.cwd()
RAW_DIR = BASE_DIR / "Data" / "raw"
PROCESSED_DIR = BASE_DIR / "Data" / "processed"
METADATA_DIR = BASE_DIR / "Data" / "metadata"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
METADATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw directory: {RAW_DIR}")
print(f"Processed directory: {PROCESSED_DIR}")
print(f"Metadata directory: {METADATA_DIR}")


Raw directory: /Users/ronny/Desktop/ML projects/DisasterOps/Data/raw
Processed directory: /Users/ronny/Desktop/ML projects/DisasterOps/Data/processed
Metadata directory: /Users/ronny/Desktop/ML projects/DisasterOps/Data/metadata


In [5]:
def get_citation_prefix(pdf_path: Path) -> str:
    """Generate citation prefix from file path"""
    category = pdf_path.parent.name
    filename = pdf_path.stem.lower()
    
    if category == "ics_forms":
        # Extract form number (e.g., "ics_form_201" -> "fema_ics201")
        match = re.search(r'ics_form_(\d+)', filename)
        if match:
            return f"fema_ics{match.group(1)}"
        elif "booklet" in filename:
            return "fema_ics_forms_booklet"
        elif "reference" in filename:
            return "fema_ics_reference"
    elif category == "cert":
        return "cert_basic_training_manual"
    elif category == "ready_gov":
        # Map filenames to citation prefixes
        if "are_you_ready" in filename:
            return "ready_gov_are_you_ready"
        elif "caregivers" in filename:
            return "ready_gov_caregivers"
        elif "earthquake" in filename:
            return "ready_gov_earthquake"
        elif "flood" in filename:
            return "ready_gov_flood"
        elif "hurricane" in filename:
            return "ready_gov_hurricane"
        elif "tornado" in filename:
            return "ready_gov_tornado"
        elif "wildfire" in filename:
            return "ready_gov_wildfire"
    elif category == "fema_other":
        if "nims_doctrine" in filename:
            return "fema_nims_doctrine"
        elif "nrf" in filename:
            return "fema_nrf"
        elif "nims" in filename:
            return f"fema_nims_{filename.replace('nims-', '').replace('_', '_')}"
    
    return f"{category}_{filename}"

def extract_sections(text: str) -> List[Dict]:
    """Extract sections based on headers and track page numbers"""
    lines = text.split('\n')
    sections = []
    current_section = {"title": "Introduction", "content": [], "page": 1}
    current_page = 1
    
    # Patterns for section headers
    header_patterns = [
        r'^(Chapter\s+\d+|Section\s+\d+|Unit\s+\d+)',
        r'^[A-Z][A-Z\s]{10,}$',  # ALL CAPS lines
        r'^\d+\.\s+[A-Z]',  # Numbered sections
        r'^[IVX]+\.\s+[A-Z]',  # Roman numerals
    ]
    
    for line in lines:
        # Extract page number from markers
        page_match = re.search(r'--- Page (\d+) ---', line)
        if page_match:
            current_page = int(page_match.group(1))
            continue
        
        line_stripped = line.strip()
        if not line_stripped:
            if current_section["content"]:
                current_section["content"].append("")
            continue
        
        # Check if line looks like a header
        is_header = any(re.match(pattern, line_stripped) for pattern in header_patterns)
        
        if is_header and len(line_stripped) < 100 and len(current_section["content"]) > 5:
            # Save current section
            sections.append({
                "title": current_section["title"],
                "content": "\n".join(current_section["content"]).strip(),
                "page": current_section["page"]
            })
            # Start new section
            current_section = {"title": line_stripped, "content": [], "page": current_page}
        else:
            current_section["content"].append(line_stripped)
            if current_section["page"] == 1 and current_page > 1:
                current_section["page"] = current_page
    
    # Add final section
    if current_section["content"]:
        sections.append({
            "title": current_section["title"],
            "content": "\n".join(current_section["content"]).strip(),
            "page": current_section["page"]
        })
    
    return sections if sections else [{"title": "Full Document", "content": text, "page": 1}]


In [6]:
def process_pdf(pdf_path: Path) -> Dict:
    """Process a single PDF and extract structured content"""
    citation_prefix = get_citation_prefix(pdf_path)
    print(f"Processing: {pdf_path.name} -> {citation_prefix}")
    
    full_text = ""
    pages_content = []
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                text = page.extract_text() or ""
                full_text += f"\n\n--- Page {page_num} ---\n\n{text}"
                pages_content.append({
                    "page": page_num,
                    "text": text
                })
    except Exception as e:
        print(f"Error processing {pdf_path.name}: {e}")
        return None
    
    # Extract sections
    sections = extract_sections(full_text)
    
    # Create citation mappings
    citations = []
    section_idx = 1
    
    for section in sections:
        start_page = section.get("page", 1)
        citation_id = f"{citation_prefix}_section{section_idx}_p{start_page}"
        citations.append({
            "citation_id": citation_id,
            "section_title": section["title"],
            "section_number": section_idx,
            "start_page": start_page,
            "content_preview": section["content"][:200] + "..." if len(section["content"]) > 200 else section["content"]
        })
        section_idx += 1
    
    # Clean full_text (remove page markers for cleaner storage)
    clean_text = re.sub(r'--- Page \d+ ---\n\n', '\n', full_text)
    
    return {
        "source_file": str(pdf_path.relative_to(BASE_DIR)),
        "citation_prefix": citation_prefix,
        "total_pages": len(pages_content),
        "full_text": clean_text,
        "pages": pages_content,
        "sections": sections,
        "citations": citations,
        "metadata": {
            "category": pdf_path.parent.name,
            "filename": pdf_path.name,
            "word_count": len(clean_text.split()),
            "char_count": len(clean_text)
        }
    }


In [7]:
# Process all PDFs
all_pdfs = list(RAW_DIR.rglob("*.pdf"))
print(f"Found {len(all_pdfs)} PDF files to process\n")

processed_docs = []
failed_docs = []

for pdf_path in all_pdfs:
    result = process_pdf(pdf_path)
    if result:
        processed_docs.append(result)
        
        # Save processed version
        processed_file = PROCESSED_DIR / f"{result['citation_prefix']}.json"
        with open(processed_file, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        
        # Save metadata
        metadata_file = METADATA_DIR / f"{result['citation_prefix']}_metadata.json"
        metadata = {
            "citation_prefix": result["citation_prefix"],
            "source_file": result["source_file"],
            "total_pages": result["total_pages"],
            "num_sections": len(result["sections"]),
            "num_citations": len(result["citations"]),
            "metadata": result["metadata"]
        }
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
    else:
        failed_docs.append(str(pdf_path))

print(f"\n✅ Processed: {len(processed_docs)}")
print(f"❌ Failed: {len(failed_docs)}")
if failed_docs:
    print("\nFailed files:")
    for f in failed_docs:
        print(f"  - {f}")


Processing: ics_form_209_incident_status_summary_v3.pdf -> fema_ics209
Processing: ics_form_205a_communications_list_v3.pdf -> fema_ics205
Processing: ics_form_211_incident_check-in_list_v3.1.pdf -> fema_ics211
Processing: ics_form_207_incident_organization_chart_v3.pdf -> fema_ics207
Processing: ics_form_213_general_message_v3.pdf -> fema_ics213
Processing: ics_form_210_resource_status_change_v3.pdf -> fema_ics210
Processing: ics_form_215_operational_planning_worksheet_v3.pdf -> fema_ics215
Processing: ics_form_204_assignment_list_v3.1.pdf -> fema_ics204
Processing: ics_form_201_incident_briefing_v3.pdf -> fema_ics201
Processing: nims-guideline-resource-management-preparedness.pdf -> fema_nims_guideline-resource-management-preparedness
Processing: nims_doctrine.pdf -> fema_nims_doctrine
Processing: nims-incident-complexity-guide.pdf -> fema_nims_incident-complexity-guide
Processing: nrf_finalapproved_2011028.pdf -> fema_nrf
Processing: cert_basic_training_manual.pdf -> cert_basic_trai

In [9]:
# Create master citation index
citation_index = {}
for doc in processed_docs:
    for citation in doc["citations"]:
        citation_index[citation["citation_id"]] = {
            "citation_prefix": doc["citation_prefix"],
            "source_file": doc["source_file"],
            "section_title": citation["section_title"],
            "section_number": citation["section_number"],
            "start_page": citation["start_page"],
            "content_preview": citation["content_preview"]
        }

# Save citation index
index_file = METADATA_DIR / "citation_index.json"
with open(index_file, 'w', encoding='utf-8') as f:
    json.dump(citation_index, f, indent=2, ensure_ascii=False)

print(f"✅ Citation index created with {len(citation_index)} entries")
print(f"📁 Saved to: {index_file}")

# Create summary statistics
summary = {
    "total_documents": len(processed_docs),
    "total_pages": sum(doc["total_pages"] for doc in processed_docs),
    "total_citations": len(citation_index),
    "categories": defaultdict(int),
    "word_count": sum(doc["metadata"]["word_count"] for doc in processed_docs)
}

for doc in processed_docs:
    summary["categories"][doc["metadata"]["category"]] += 1

summary_file = METADATA_DIR / "processing_summary.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\n📊 Summary:")
print(f"  Documents: {summary['total_documents']}")
print(f"  Pages: {summary['total_pages']}")
print(f"  Citations: {summary['total_citations']}")
print(f"  Words: {summary['word_count']:,}")
print(f"  Categories: {dict(summary['categories'])}")


✅ Citation index created with 754 entries
📁 Saved to: /Users/ronny/Desktop/ML projects/DisasterOps/Data/metadata/citation_index.json

📊 Summary:
  Documents: 49
  Pages: 2714
  Citations: 754
  Words: 541,509
  Categories: {'ready_gov': 7, 'ics_forms': 21, 'fema_other': 4, 'cert': 1, '04 IG': 4, '06 Handouts': 6, '03 SM': 2, 'Handouts': 1, 'POI': 1, 'Instructor Guide': 1, 'Student Manual': 1}


### RAG PIPELINE ( Chunking + embeddings)

In [11]:
!pip install sentence-transformers rank-bm25 torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sentence-transformers]


In [12]:
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import pickle
from typing import List, Dict, Tuple

# Paths
CHUNKS_DIR = BASE_DIR / "Data" / "chunks"
EMBEDDINGS_DIR = BASE_DIR / "Data" / "embeddings"
CHUNKS_DIR.mkdir(parents=True, exist_ok=True)
EMBEDDINGS_DIR.mkdir(parents=True, exist_ok=True)

# Load embedding model (using sentence-transformers as default - no API key needed)
print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, good quality, 384 dims
print("✓ Model loaded")


Loading embedding model...
✓ Model loaded


In [13]:
def chunk_text(text: str, citation_id: str, chunk_size: int = 500, chunk_overlap: int = 50) -> List[Dict]:
    """Create chunks with overlap and preserve citation mapping"""
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - chunk_overlap):
        chunk_words = words[i:i + chunk_size]
        chunk_text = " ".join(chunk_words)
        
        if chunk_text.strip():
            chunks.append({
                "text": chunk_text,
                "citation_id": citation_id,
                "chunk_index": len(chunks),
                "word_count": len(chunk_words)
            })
    
    return chunks

def process_documents_for_rag(processed_docs: List[Dict]) -> Tuple[List[Dict], Dict]:
    """Create chunks from all processed documents"""
    all_chunks = []
    chunk_to_citation = {}
    
    for doc in processed_docs:
        citation_prefix = doc["citation_prefix"]
        
        # Chunk by sections (hierarchical approach)
        for section in doc["sections"]:
            section_citation = f"{citation_prefix}_section{section.get('section_number', 1)}_p{section.get('page', 1)}"
            section_text = section["content"]
            
            # Further chunk if section is too long
            if len(section_text.split()) > 500:
                section_chunks = chunk_text(section_text, section_citation)
                all_chunks.extend(section_chunks)
            else:
                all_chunks.append({
                    "text": section_text,
                    "citation_id": section_citation,
                    "chunk_index": 0,
                    "word_count": len(section_text.split())
                })
    
    # Build citation mapping
    for i, chunk in enumerate(all_chunks):
        chunk_to_citation[i] = chunk["citation_id"]
    
    return all_chunks, chunk_to_citation


In [14]:
# Load processed documents and create chunks
print("Loading processed documents...")
processed_files = list(PROCESSED_DIR.glob("*.json"))
processed_docs = []
for f in processed_files:
    with open(f, 'r', encoding='utf-8') as file:
        processed_docs.append(json.load(file))

print(f"Loaded {len(processed_docs)} documents")

print("\nCreating chunks...")
all_chunks, chunk_to_citation = process_documents_for_rag(processed_docs)

print(f"✓ Created {len(all_chunks)} chunks")
print(f"✓ Average chunk size: {np.mean([c['word_count'] for c in all_chunks]):.0f} words")

# Save chunks
chunks_file = CHUNKS_DIR / "all_chunks.json"
with open(chunks_file, 'w', encoding='utf-8') as f:
    json.dump(all_chunks, f, indent=2, ensure_ascii=False)

citation_map_file = CHUNKS_DIR / "chunk_to_citation.json"
with open(citation_map_file, 'w', encoding='utf-8') as f:
    json.dump(chunk_to_citation, f, indent=2)

print(f"✓ Saved chunks to {chunks_file}")


Loading processed documents...
Loaded 47 documents

Creating chunks...
✓ Created 1694 chunks
✓ Average chunk size: 343 words
✓ Saved chunks to /Users/ronny/Desktop/ML projects/DisasterOps/Data/chunks/all_chunks.json


In [15]:
# Generate embeddings
print("Generating embeddings...")
chunk_texts = [chunk["text"] for chunk in all_chunks]
embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True, batch_size=32)

print(f"✓ Generated {len(embeddings)} embeddings")
print(f"✓ Embedding dimension: {embeddings.shape[1]}")

# Save embeddings
embeddings_file = EMBEDDINGS_DIR / "embeddings.npy"
np.save(embeddings_file, embeddings)
print(f"✓ Saved embeddings to {embeddings_file}")


Generating embeddings...


Batches:   0%|          | 0/53 [00:00<?, ?it/s]

✓ Generated 1694 embeddings
✓ Embedding dimension: 384
✓ Saved embeddings to /Users/ronny/Desktop/ML projects/DisasterOps/Data/embeddings/embeddings.npy


In [16]:
# Build BM25 index (sparse retrieval)
print("Building BM25 index...")
tokenized_chunks = [chunk["text"].lower().split() for chunk in all_chunks]
bm25 = BM25Okapi(tokenized_chunks)

print("✓ BM25 index built")

# Save BM25 model
bm25_file = EMBEDDINGS_DIR / "bm25_model.pkl"
with open(bm25_file, 'wb') as f:
    pickle.dump(bm25, f)
print(f"✓ Saved BM25 model to {bm25_file}")


Building BM25 index...
✓ BM25 index built
✓ Saved BM25 model to /Users/ronny/Desktop/ML projects/DisasterOps/Data/embeddings/bm25_model.pkl


In [17]:
def hybrid_search(query: str, embeddings: np.ndarray, bm25: BM25Okapi, chunk_texts: List[str], 
                  top_k: int = 10, dense_weight: float = 0.5) -> List[Tuple[int, float]]:
    """Hybrid search combining dense (embeddings) and sparse (BM25) retrieval"""
    # Dense search
    query_embedding = embedding_model.encode([query])[0]
    dense_scores = np.dot(embeddings, query_embedding)
    dense_scores = (dense_scores - dense_scores.min()) / (dense_scores.max() - dense_scores.min() + 1e-8)
    
    # Sparse search (BM25)
    tokenized_query = query.lower().split()
    sparse_scores = bm25.get_scores(tokenized_query)
    sparse_scores = (sparse_scores - sparse_scores.min()) / (sparse_scores.max() - sparse_scores.min() + 1e-8)
    
    # Combine scores
    hybrid_scores = dense_weight * dense_scores + (1 - dense_weight) * sparse_scores
    
    # Get top k
    top_indices = np.argsort(hybrid_scores)[::-1][:top_k]
    results = [(int(idx), float(hybrid_scores[idx])) for idx in top_indices]
    
    return results

# Test search
test_query = "How to establish triage area during emergency"
print(f"Testing search with query: '{test_query}'")
results = hybrid_search(test_query, embeddings, bm25, chunk_texts, top_k=5)

print("\nTop 5 results:")
for i, (idx, score) in enumerate(results, 1):
    chunk = all_chunks[idx]
    print(f"\n{i}. Score: {score:.4f}")
    print(f"   Citation: {chunk['citation_id']}")
    print(f"   Text: {chunk['text'][:150]}...")


Testing search with query: 'How to establish triage area during emergency'

Top 5 results:

1. Score: 0.9291
   Citation: fema_nims_doctrine_section1_p29
   Text: incident management frequently necessitates ratios significantly different from this. The 1:5 ratio is a guideline, and incident personnel use their b...

2. Score: 0.9291
   Citation: Handouts_national_incident_management system_third edition_october_2017_section1_p29
   Text: incident management frequently necessitates ratios significantly different from this. The 1:5 ratio is a guideline, and incident personnel use their b...

3. Score: 0.9026
   Citation: ready_gov_caregivers_section1_p17
   Text: Sometimes the best way to stay safe in an emergency is to get inside and stay inside a building or vehicle until danger has passed. This is also known...

4. Score: 0.8836
   Citation: fema_ics_reference_section1_p13
   Text: A temporary location managed by the Operations Section. A staging area can be any location in which pers

In [18]:
# Save RAG pipeline metadata
rag_metadata = {
    "total_chunks": len(all_chunks),
    "embedding_dim": embeddings.shape[1],
    "embedding_model": "all-MiniLM-L6-v2",
    "chunk_size": 500,
    "chunk_overlap": 50,
    "files": {
        "chunks": str(chunks_file.relative_to(BASE_DIR)),
        "embeddings": str(embeddings_file.relative_to(BASE_DIR)),
        "bm25": str(bm25_file.relative_to(BASE_DIR)),
        "citation_map": str(citation_map_file.relative_to(BASE_DIR))
    }
}

rag_metadata_file = EMBEDDINGS_DIR / "rag_metadata.json"
with open(rag_metadata_file, 'w', encoding='utf-8') as f:
    json.dump(rag_metadata, f, indent=2)

print(f"\n✅ RAG Pipeline Complete!")
print(f"📊 Summary:")
print(f"   Chunks: {rag_metadata['total_chunks']}")
print(f"   Embeddings: {embeddings.shape}")
print(f"   BM25 index: Built")
print(f"   Metadata saved to: {rag_metadata_file}")



✅ RAG Pipeline Complete!
📊 Summary:
   Chunks: 1694
   Embeddings: (1694, 384)
   BM25 index: Built
   Metadata saved to: /Users/ronny/Desktop/ML projects/DisasterOps/Data/embeddings/rag_metadata.json
